# Gravity

 This tutorial demonstrates how to simulate gravitational attraction between two particles using the **SOFA** physics engine.

Initialization:

In [ ]:
import Sofa
import SofaRuntime
SofaRuntime.init()

In [ ]:
root = Sofa.Core.Node("root")

In [ ]:
print(f"By default, SOFA gravity is set to {root.gravity.value}")

By default, SOFA applies uniform gravity. Here, we disable it to focus on **non-uniform gravitational forces** between particles.

In [ ]:
root.gravity.value = [0, 0, 0]

We use SOFA's built-in animation loop for time-stepped physics updates.

In [ ]:
root.addObject("DefaultAnimationLoop")

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.ODESolver.Forward")
root.addObject("EulerExplicitSolver")

We create two particles

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.StateContainer")
initial_position = [[0,0], [1,-1]]
initial_velocity = [[0,0], [0,1.5]]
root.addObject("MechanicalObject", template="Vec2", name="particles", 
    position=initial_position, velocity=initial_velocity)

Definition of the mass of the particles:

In [ ]:
mass = [1, 1]

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.Mass")
root.addObject("UniformMass", template="Vec2", name="mass", vertexMass=mass)

We implement a gravitational force between the two particles using Newton’s law of gravitation:

$$
F_0 = G \frac{m_0 m_1}{r^2} \hat{r}
$$

where:

- $G$ is the gravitational constant (6.67430 x 10-11 m3 kg-1 s-2)
- $m_0$, $m_1$ are particle masses
- $\hat{r}$ is the unit vector pointing from particle 1 to particle 0


In [ ]:
import numpy

# gravitational constant
G = 1

class GravityForceField(Sofa.Core.ForceFieldVec2d):

    def addForce(self, m, out_force, pos, vel):

        d = pos[0] - pos[1] # vector from particle 1 to particle 0
        r_2 = numpy.dot(d, d) # squared norm

        d = d / numpy.sqrt(r_2) #make d a unit vector
        
        with out_force.writeableArray() as wa:
            wa[0] += -(G * mass[0] * mass[1] / r_2) * d
            wa[1] += (G * mass[0] * mass[1] / r_2) * d

root.addObject(GravityForceField())

In [ ]:
Sofa.Simulation.initRoot(root)

Run the simulation and record the trajectories:

In [ ]:
from matplotlib import pyplot as plt 

x = [[], []]
y = [[], []]

def retrieve_position():
    position = root.particles.position.value
    assert len(position) == 2

    x[0].append(position[0][0])
    y[0].append(position[0][1])

    x[1].append(position[1][0])
    y[1].append(position[1][1])


retrieve_position()

for iteration in range(500):
    Sofa.Simulation.animate(root, root.dt.value)
    retrieve_position()

Create the animation of the trajectory:

In [ ]:
import matplotlib.animation as animation

fig, ax = plt.subplots()
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_title("Gravity simulation using SOFA")

# Initialize empty lines for particles
line0, = ax.plot([], [], 'ro', markersize=8)  # Red circle for particle 0
line1, = ax.plot([], [], 'b-', linewidth=2)   # Blue line for particle 1

# Set initial data ranges
xmin, xmax = min(min(x[0]), min(x[1])), max(max(x[0]), max(x[1]))
ymin, ymax = min(min(y[0]), min(y[1])), max(max(y[0]), max(y[1]))

# Add margin
xmin = xmin - (xmax - xmin) * 0.1
xmax = xmax + (xmax - xmin) * 0.1
ymin = ymin - (ymax - ymin) * 0.1
ymax = ymax + (ymax - ymin) * 0.1

# Adjust axis limits to show all data
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

def animate(i):
    line0.set_xdata(x[0][:i])
    line0.set_ydata(y[0][:i])

    line1.set_xdata(x[1][:i])
    line1.set_ydata(y[1][:i])

    return (line0, line1)

# Create animation
ani = animation.FuncAnimation(
    fig=fig,
    func=animate,
    frames=len(x[0]),
    interval=10  # milliseconds between frames
)
plt.close()


Visualize the animation as javascript:

In [ ]:
from IPython.display import HTML
html = ani.to_jshtml()
HTML(html)

Visualize the animation as html video (requires ffmpeg):

In [ ]:
# from IPython.display import HTML
# video = ani.to_html5_video()
# HTML(video)